In [1]:
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
import numpy as np
from sklearn.datasets import make_classification

# Generate a random multi-output classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=3, n_clusters_per_class=1, random_state=42)

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def custom_metric(beta):
    def custom_metric_fn(y_true, y_pred):
        auc = tf.keras.metrics.AUC()(y_true, y_pred)
        tp = tf.cast(tf.math.count_nonzero(tf.logical_and(tf.equal(y_true, 1), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
        fp = tf.cast(tf.math.count_nonzero(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
        fn = tf.cast(tf.math.count_nonzero(tf.logical_and(tf.equal(y_true, 1), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
        tn = tf.cast(tf.math.count_nonzero(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))

        numerator = (beta * auc + tp + fp + fn) / (tp + fn + fp + tn)
        denominator = 1 + beta
        custom_metric_value = numerator / denominator

        return custom_metric_value

    return custom_metric_fn

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 3 output classes
])

beta_value = 0.5
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[custom_metric(beta=beta_value)])

In [6]:
y_train_one_hot = tf.one_hot(y_train, depth=3)  # Assuming 3 output classes
y_test_one_hot = tf.one_hot(y_test, depth=3)

In [ ]:
c

In [10]:
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# Evaluate the model on the test set
loss, custom_metric_value = model.evaluate(X_test, y_test_one_hot)
print(f"Test Loss: {loss}, Custom Metric Value: {custom_metric_value}")

Epoch 1/10


TypeError: 'NoneType' object is not callable